# 助詞穴埋めタスクGRPO学習ノートブック（改良版）

training_fill.ipynbのログ分析結果に基づいて、報酬バランスを改善したバージョンです。

## 主な改善点
1. **報酬の再バランス**: 正解率95%という高すぎる報酬を調整
2. **部分点の導入**: より繊細な評価で学習を促進
3. **文法説明の重視**: reasoning部分の質を詳細に評価
4. **ペナルティの強化**: フォーマットエラーや不正解により厳しいペナルティ

## 特徴
- nihongo-dojoライブラリの改良版報酬関数を使用
- より効果的なGRPO学習のための報酬設計
- 拡張ログ機能で詳細な分析が可能

## 1. 環境セットアップ

In [1]:
# GPU環境の確認
import torch
import os

print("=== GPU環境チェック ===")
print(f"CUDA利用可能: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"GPUメモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print("✅ GPU環境が正常に検出されました！")
else:
    print("❌ GPUが検出されません！")
    print("上記の手順でGPUを有効にしてください。")
    print("その後、ランタイムを再起動してこのセルを再実行してください。")

# Colab環境かチェック
if 'COLAB_GPU' in os.environ:
    print(f"\nGoogle Colab GPU: {os.environ['COLAB_GPU']}")
elif 'COLAB_' in "".join(os.environ.keys()):
    print("\nGoogle Colab環境です。GPUを有効にしてください。")
else:
    print("\nローカル環境で実行中")

=== GPU環境チェック ===
CUDA利用可能: True
GPU名: Tesla T4
GPUメモリ: 14.74 GB
✅ GPU環境が正常に検出されました！

Google Colab GPU: 1


*   unsloth 2025.7.3
*   unsloth-zoo 2025.7.4

の組み合わせで動作。

https://github.com/unslothai/unsloth/issues/2983

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth==2025.7.3 unsloth-zoo==2025.7.4 vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth==2025.7.3 unsloth-zoo==2025.7.4 vllm==0.8.5.post1

In [3]:
#@title Colab追加インストール { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth==2025.7.3, unsloth-zoo==2025.7.4 vllm
else:
    !pip install --no-deps unsloth==2025.7.3 vllm==0.8.5.post1
    # Qwen3_(4B)_GRPO.ipynbと同じ設定
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth-zoo==2025.7.4
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLMはnumpyを再インストールするためColabを壊す
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

## 2. モデルのロード

In [4]:
# GPU環境が確認できた場合のみ実行
if not torch.cuda.is_available():
    raise RuntimeError("GPUが検出されません。上記の手順でGPUを有効にしてください。")

from unsloth import FastLanguageModel
import torch

print("モデルをロード中...")
max_seq_length = 2048
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-18 04:08:43 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-18 04:08:43 [__init__.py:239] Automatically detected platform cuda.
モデルをロード中...
==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.53.2. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill t

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.7.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


## 3.nihongo-dojoのインストール

In [9]:
# Google Colab環境での準備
%cd /content
!unzip nihongo-dojo.zip
!pip install japanize-matplotlib scikit-learn
%cd /content/nihongo-dojo/
!pip install -e .
%cd /content/

/content
Archive:  nihongo-dojo.zip
replace __MACOSX/._nihongo-dojo? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/._nihongo-dojo  
  inflating: nihongo-dojo/.DS_Store  
  inflating: __MACOSX/nihongo-dojo/._.DS_Store  
  inflating: nihongo-dojo/requirements.txt  
  inflating: __MACOSX/nihongo-dojo/._requirements.txt  
  inflating: nihongo-dojo/pyproject.toml  
  inflating: __MACOSX/nihongo-dojo/._pyproject.toml  
  inflating: __MACOSX/nihongo-dojo/.___pycache__  
  inflating: nihongo-dojo/README.md  
  inflating: __MACOSX/nihongo-dojo/._README.md  
  inflating: nihongo-dojo/setup.py   
  inflating: __MACOSX/nihongo-dojo/._setup.py  
  inflating: __MACOSX/nihongo-dojo/._scripts  
  inflating: __MACOSX/nihongo-dojo/._nihongo_dojo  
  inflating: __MACOSX/nihongo-dojo/._notebooks  
  inflating: __MACOSX/nihongo-dojo/._nihongo_dojo.egg-info  
  inflating: nihongo-dojo/CLAUDE.md  
  inflating: __MACOSX/nihongo-dojo/._CLAUDE.md  
  inflating: nihongo-dojo/__pycache__/core.cpyt

In [10]:
import sys, importlib
module_path = "/content/nihongo-dojo"
if module_path not in sys.path:
    sys.path.append(module_path)

import nihongo_dojo
importlib.reload(nihongo_dojo)
from nihongo_dojo.colab import TrainingLogger

/content/nihongo-dojo/nihongo_dojo/__init__.py:149: UserWarning: Optional colab module not available: cannot import name 'plot_training_history' from 'nihongo_dojo.colab' (/content/nihongo-dojo/nihongo_dojo/colab/__init__.py)
  warnings.warn(f"Optional colab module not available: {e}")


## 4. チャットテンプレートの設定

In [12]:
# チャットテンプレートを設定
system_prompt = "あなたは親切で賢いアシスタントです。日本語の助詞を正しく選んで文を完成させてください。"

# デフォルト設定
reasoning_start = "<reasoning>"
reasoning_end = "</reasoning>"
solution_start = "<answer>"
solution_end = "</answer>"

chat_template = """{% if messages[0]['role'] == 'system' %}{{ messages[0]['content'] }}{% endif %}

{% for message in messages %}{% if message['role'] == 'user' %}
User: {{ message['content'] }}

{% elif message['role'] == 'assistant' %}{{ 'Assistant: ' + message['content'] }}{% endif %}{% endfor %}"""

tokenizer.chat_template = chat_template

## 5. データセットの作成

In [13]:
# 助詞穴埋めデータセット生成
import os
if not os.path.exists("./datasets/nihongo-dojo-particle-fill/"):
    %cd /content
    !python nihongo-dojo/scripts/generate_datasets.py --tasks PARTICLE_FILL --custom-size 2000 --output-format jsonl --output-dir ./datasets
else:
    print("データセットは既に生成済みです")

/content
/content/nihongo-dojo/nihongo_dojo/__init__.py:149: UserWarning: Optional colab module not available: cannot import name 'plot_training_history' from 'nihongo_dojo.colab' (/content/nihongo-dojo/nihongo_dojo/colab/__init__.py)
  warnings.warn(f"Optional colab module not available: {e}")
🎯 カスタムデータセット生成:
   タスク: particle_fill
💾 データセット保存中: ./datasets/nihongo-dojo-particle_fill
💾 データセット保存中: ./datasets/nihongo-dojo-particle_fill
✅ データセット保存完了: ./datasets/nihongo-dojo-particle_fill
✅ データセット生成完了: ./datasets/nihongo-dojo-particle_fill.jsonl


In [14]:
from datasets import load_dataset, Dataset
import numpy as np
import sys

print("データセットを読み込み中...")

dataset_path = './datasets/nihongo-dojo-particle_fill/'

# all.jsonlファイルが存在するか確認
import os
if os.path.exists(os.path.join(dataset_path, 'all.jsonl')):
    # 直接ファイルを読み込む方法
    import json
    data = []
    with open(os.path.join(dataset_path, 'all.jsonl'), 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    dataset = Dataset.from_list(data)
elif os.path.exists(os.path.join(dataset_path, 'train.jsonl')):
    # train.jsonlを試す
    import json
    data = []
    with open(os.path.join(dataset_path, 'train.jsonl'), 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    dataset = Dataset.from_list(data)
else:
    # データセットが見つからない場合のエラーメッセージ
    raise FileNotFoundError(f"データセットが見つかりません: {dataset_path}")

print(f"データセットサイズ: {len(dataset)}")

# データセットの例を表示
print("\nデータセットの例:")
for i in range(min(3, len(dataset))):
    print(f"\n例{i+1}:")
    print(f"  問題: {dataset[i]['instruction']}{dataset[i]['input']}")
    print(f"  答え: {dataset[i]['answer']}")
    print(f"  説明: {dataset[i]['think'][:100]}...")

データセットを読み込み中...
データセットサイズ: 2000

データセットの例:

例1:
  問題: 文中の[　]に入る適切な助詞を答えてください。原則[　]則って判断する。
  答え: に
  説明: この文では基準を表す助詞を使います。 答えは「に」です。...

例2:
  問題: 文中の[　]に入る適切な助詞を答えてください。原則[　]則って判断する。
  答え: に
  説明: この文では基準を表す助詞を使います。 答えは「に」です。...

例3:
  問題: 文中の[　]に入る適切な助詞を答えてください。原則[　]則って判断する。
  答え: に
  説明: この文では基準を表す助詞を使います。 答えは「に」です。...


In [15]:
# フォーマット変換
formatted_data = []
for item in dataset:
    question = item['instruction'] + item['input']
    answer = item['answer']
    think = item['think']
    solution = f"{reasoning_start}\n{think}\n{reasoning_end}\n{solution_start}{answer}{solution_end}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
        {"role": "assistant", "content": solution}
    ]

    formatted_data.append({
        "Messages": messages,
        "problem": question,
        "solution": solution,
        "answer": answer,
    })

dataset = Dataset.from_list(formatted_data)
print(f"フォーマット済みデータセット: {len(dataset)}個")

フォーマット済みデータセット: 2000個


## 6. SFTによる事前学習（フォーマット学習）

コールドスタート問題を解消するために、`<reasoning></reasoning><answer></answer>`のフォーマットで出力ができるようにSFTで学習させます。

In [16]:
# 短い例のみを選択
dataset = dataset.map(lambda x: {"N": len(tokenizer.apply_chat_template(x["Messages"]))})
pre_train_dataset = dataset.filter(lambda x: x["N"] <= max_seq_length/2).select(range(min(50, len(dataset))))
pre_train_dataset = pre_train_dataset.map(lambda x: {"text": tokenizer.apply_chat_template(x["Messages"], tokenize=False)})

print(f"事前学習データセット: {len(pre_train_dataset)}個")

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

事前学習データセット: 50個


In [17]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = pre_train_dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

print("フォーマット学習を開始...")
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

フォーマット学習を開始...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 2 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
5,1.638200
10,0.378700
15,0.253600
20,0.358400
25,0.206400
30,0.199700
35,0.194600
40,0.080100
45,0.143500
50,0.082700


TrainOutput(global_step=100, training_loss=0.2080068953335285, metrics={'train_runtime': 56.171, 'train_samples_per_second': 1.78, 'train_steps_per_second': 1.78, 'total_flos': 222995415373824.0, 'train_loss': 0.2080068953335285})

## 7. ログ関連

In [18]:
from nihongo_dojo.colab import TrainingLogger

# ログ管理インスタンスを作成（詳細ログも有効化）
# タスク名を指定してログファイル名を設定
logger = TrainingLogger(log_dir="./logs", task_name="fill", enable_detailed_logging=True)

# グローバル変数（互換性のため）
global TRAINING_LOGS, REWARD_LOGS, ACCURACY_STATS
TRAINING_LOGS = logger.training_logs
REWARD_LOGS = logger.reward_logs
ACCURACY_STATS = {
    'correct_format': [],
    'correct_answer': [],
    'partial_answer': [],
    'wrong_answer': [],
    'no_answer': []
}

ログファイル: ./logs/kanji_training_20250718_042854.jsonl
学習履歴ファイル: ./logs/kanji_training_20250718_042854_history.jsonl
詳細ログファイル: ./logs/kanji_training_20250718_042854_detailed.jsonl
※ 学習履歴は1ステップごとに自動保存されます


## 8. 改良版GRPO報酬関数を定義

In [ ]:
# 改良版の報酬関数を定義（データの偏りに対応）
import re
from collections import Counter

# 基本報酬関数をインポート
from nihongo_dojo.reward import JapaneseTaskRewardFunctions

# 基本報酬関数インスタンスを作成
base_reward_functions = JapaneseTaskRewardFunctions(
    reasoning_start=reasoning_start,
    reasoning_end=reasoning_end,
    solution_start=solution_start,
    solution_end=solution_end,
    eos_token=tokenizer.eos_token
)

# 答えの分布を記録
answer_distribution = Counter()

def improved_check_particle(prompts=None, completions=None, completion_ids=None, answer=None, **kwargs):
    """助詞穴埋めタスクに特化した改良版報酬関数"""
    global answer_distribution
    
    # 基本的な答え抽出
    if prompts is not None and completions is not None:
        responses = completions
    else:
        responses = kwargs.get('completions', [])
    
    # 答えの処理
    if answer and isinstance(answer, list) and len(answer) > 0 and isinstance(answer[0], str) and '<answer>' in answer[0]:
        extracted_answers = []
        for ans in answer:
            match = re.search(r'<answer>(.+?)</answer>', ans, re.DOTALL)
            if match:
                extracted_answers.append(match.group(1).strip())
            else:
                extracted_answers.append(ans)
        answer = extracted_answers
    
    if not isinstance(answer, list):
        answer = [answer] * len(responses)
    
    # フォーマット抽出
    match_format = re.compile(
        rf"{reasoning_end}.*?"
        rf"{solution_start}(.+?){solution_end}"
        rf".*$",
        flags=re.MULTILINE | re.DOTALL
    )
    
    extracted_responses = []
    for r in responses:
        if isinstance(r, str):
            text = r
        elif isinstance(r, list) and len(r) > 0:
            text = r[0].get("content", "") if isinstance(r[0], dict) else str(r[0])
        else:
            text = ""
        
        match = match_format.search(text)
        if match:
            extracted_responses.append(match.group(1).strip())
        else:
            extracted_responses.append(None)
    
    scores = []
    
    # 頻出助詞のペナルティ係数（「に」が多すぎる問題への対処）
    frequency_penalty = {
        "に": 0.8,  # 頻出のためペナルティ
        "が": 0.9,
        "を": 0.9,
        "で": 1.0,
        "と": 1.0,
        "へ": 1.1,  # 少ない助詞にボーナス
        "から": 1.1,
        "まで": 1.1,
        "より": 1.2,
    }
    
    for guess, true_answer in zip(extracted_responses, answer):
        # 答えの分布を記録
        if true_answer:
            answer_distribution[true_answer] += 1
        
        # フォーマットエラー
        if guess is None:
            scores.append(-3.0)
            continue
        
        # 頻度に基づくペナルティ/ボーナスを適用
        penalty = frequency_penalty.get(true_answer, 1.0)
        
        # 完全一致
        if guess == true_answer:
            scores.append(2.0 * penalty)
            continue
        
        # 助詞のカテゴリー別評価
        particle_categories = {
            "格助詞": ["が", "を", "に", "で", "と", "へ", "から", "まで", "より"],
            "接続助詞": ["ば", "と", "ても", "けど", "から", "ので", "のに"],
            "副助詞": ["は", "も", "こそ", "さえ", "でも", "しか", "ばかり"],
            "終助詞": ["か", "ね", "よ", "な", "の"],
        }
        
        # カテゴリー内の混同チェック
        guess_category = None
        answer_category = None
        
        for category, particles in particle_categories.items():
            if guess in particles:
                guess_category = category
            if true_answer in particles:
                answer_category = category
        
        if guess_category == answer_category and guess_category is not None:
            # 同じカテゴリー内の混同
            if guess_category == "格助詞":
                # 格助詞の混同は特に評価を細かく
                if (guess, true_answer) in [("に", "で"), ("で", "に")]:
                    scores.append(0.3)  # 場所の「に」と手段の「で」
                elif (guess, true_answer) in [("が", "を"), ("を", "が")]:
                    scores.append(-0.5)  # 主格と対格の混同は重大
                else:
                    scores.append(0.0)
            else:
                scores.append(0.5)  # 他のカテゴリー内混同
        else:
            # 異なるカテゴリー間の混同
            scores.append(-1.5)
    
    return scores

def enhanced_particle_reasoning(prompts=None, completions=None, completion_ids=None, answer=None, **kwargs):
    """助詞選択の理由説明を評価する報酬関数"""
    if prompts is not None and completions is not None:
        responses = completions
    else:
        responses = kwargs.get('completions', [])
    
    scores = []
    
    # 理由説明のパターン
    good_patterns = [
        r"対象を表す",
        r"場所を表す",
        r"手段を表す",
        r"方向を表す",
        r"基準を表す",
        r"主語を表す",
        r"目的語を表す",
        r"格助詞",
        r"接続助詞",
        r"副助詞",
    ]
    
    for r in responses:
        if isinstance(r, str):
            text = r
        elif isinstance(r, list) and len(r) > 0:
            text = r[0].get("content", "") if isinstance(r[0], dict) else str(r[0])
        else:
            text = ""
        
        # reasoning部分を抽出
        reasoning_match = re.search(
            rf"{reasoning_start}(.+?){reasoning_end}",
            text,
            re.DOTALL
        )
        
        if not reasoning_match:
            scores.append(-1.0)
            continue
        
        reasoning = reasoning_match.group(1)
        
        # 良い説明のパターンをチェック
        pattern_count = sum(1 for pattern in good_patterns if re.search(pattern, reasoning))
        
        if pattern_count >= 2:
            scores.append(1.0)
        elif pattern_count == 1:
            scores.append(0.5)
        else:
            scores.append(0.0)
    
    return scores

print("改良版報酬関数:")
print("1. base_reward_functions.strict_format_check - 厳格なフォーマットチェック")
print("2. improved_check_particle - 改良版助詞チェック（頻度調整付き）")
print("3. enhanced_particle_reasoning - 理由説明の評価")

In [ ]:
# nihongo_dojoライブラリのログ機能を使用
from nihongo_dojo.colab import LoggingRewardWrapper

# グローバル変数（後方互換性のため）
global PRINTED_TIMES, PRINT_EVERY_STEPS
PRINTED_TIMES = 0
PRINT_EVERY_STEPS = 5

# ログ付き報酬関数を作成（改良版を使用）
check_particle_with_logging = LoggingRewardWrapper(
    reward_func=improved_check_particle,
    logger=logger,
    print_every_steps=PRINT_EVERY_STEPS
)

## 9. 可視化コールバックの設定

In [21]:
# nihongo_dojoライブラリの可視化コールバックを使用
from nihongo_dojo.colab import GRPOVisualizationCallback

# 可視化コールバックを作成
visualization_callback = GRPOVisualizationCallback(
    update_frequency=5,
    keep_history_steps=20,
    log_filename=logger.log_filename,
    logger=logger
)

日本語フォントの設定が完了しました


## 10. GRPO学習の実行（改良版報酬関数使用）

In [22]:
# GRPO用にフォーマット
dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["problem"]},
    ],
    "answer": x["solution"],  # フルソリューションを保持（報酬関数側で実際の答えを抽出）
    "actual_answer": x["answer"],  # 実際の答えも保持
})

# プロンプト長でフィルタリング
tokenized = dataset.map(
    lambda x: {"tokens": tokenizer.apply_chat_template(x["prompt"], add_generation_prompt=True, tokenize=True)},
    batched=True,
)
tokenized = tokenized.map(lambda x: {"L": len(x["tokens"])})
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print(f"最大プロンプト長: {maximum_length}")

dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
print(f"フィルタ後のデータセット: {len(dataset)}個")

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

最大プロンプト長: 60
フィルタ後のデータセット: 1838個


In [23]:
max_prompt_length = maximum_length + 1 # + 1 念のため
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 4,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    max_steps = 2000,
    save_steps = 100,
    report_to = "none",
    output_dir = "outputs_particle_balanced",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        base_reward_functions.strict_format_check,      # 厳格なフォーマットチェック
        check_particle_with_logging,                    # 改良版助詞チェック（ログ付き）
        enhanced_particle_reasoning,                    # 理由説明の評価
    ],
    args = training_args,
    train_dataset = dataset,
    callbacks=[visualization_callback],  # ビジュアライゼーションコールバックを追加
)

print("🌸 助詞穴埋め学習のGRPO学習を開始します（改良版報酬関数使用）...")
print("📊 リアルタイムでグラフと統計情報が表示されます")
print("💡 データの偏りに対応した報酬関数で効果的な学習を実現します")
print("-"*80)
trainer.train()

## 11. モデルの評価

In [ ]:
# LoRAモデルを保存
model.save_lora("grpo_particle_balanced_lora")
print("改良版モデルを保存しました")

In [ ]:
# 助詞穴埋めテスト
test_questions = [
    "私（　）学校に行きます。",
    "本（　）読みます。",
    "友達（　）遊びます。",
    "電車（　）東京へ行きます。",
    "ペン（　）字を書きます。",
    "公園（　）走ります。",
    "母（　）プレゼントをもらいました。",
    "日本（　）アメリカは遠いです。",
    "朝ご飯（　）食べましたか？",
    "これ（　）私の本です。",
]

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)

print("="*80)
print("🗾 助詞穴埋めテスト（改良版モデル）")
print("="*80)

for i, question in enumerate(test_questions, 1):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    output = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_particle_balanced_lora"),
    )[0].outputs[0].text

    print(f"\n{i}. 問題: {question}")
    print(f"   応答: {output}")
    print("-"*40)

## 12. 学習ログの分析

In [ ]:
# nihongo_dojoライブラリの可視化関数を使用
from nihongo_dojo.colab.visualization import plot_training_history

# 学習履歴を可視化
plot_training_history(logger.history_filename)

In [ ]:
# 拡張ログ分析
from nihongo_dojo.colab import analyze_training_logs

print("\n📊 拡張ログ分析を実行中...")
analyze_training_logs(logger.log_filename)

## 13. 改善の効果を比較

改良版の報酬関数により、以下の改善が期待されます：

1. **報酬分布の改善**: 正解率95%→より適切な分布へ
2. **学習の進行**: 単調な報酬から、段階的な改善へ
3. **品質の向上**: 文法説明の質が向上
4. **エラーの減少**: フォーマットエラーの削減

In [ ]:
# 報酬分布の分析
print("\n📊 報酬分布の分析")
print("改良版の報酬関数により、以下の変化が期待されます：")
print("\n【従来版】")
print("- 2.0 (完全正解): 94.7%")
print("- -0.5 (不正解): 3.3%")
print("- その他: 2.0%")
print("- データの偏り: 「に」が答えの大半を占める")
print("\n【改良版（期待値）】")
print("- 1.6-2.4 (完全正解・頻度調整): 45-55%")
print("- 0.3-0.5 (同カテゴリー混同): 20-25%")
print("- 0.0 (格助詞内混同): 10-15%")
print("- -0.5--1.5 (異カテゴリー混同): 10-15%")
print("- -3.0 (フォーマットエラー): <5%")
print("\n特に、頻出助詞「に」への過度な学習を抑制し、多様な助詞の使い分けを促進します。")

# 答えの分布を表示
if answer_distribution:
    print("\n📊 答えの分布:")
    total = sum(answer_distribution.values())
    for particle, count in answer_distribution.most_common(10):
        percentage = (count / total) * 100
        print(f"  {particle}: {count}回 ({percentage:.1f}%)")

## 14. モデルの保存（オプション）